# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [23]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, fbeta_score, scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, HashingVectorizer
from sklearn.multioutput import MultiOutputClassifier
import pickle
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_relief_KT.db')
df = pd.read_sql('SELECT * FROM disaster_relief_KT', engine)
X = df.filter(items = ['id', 'message', 'original', 'genre'])
Y = df.drop(['id', 'message', 'original', 'genre', 'child_alone'], axis=1)
Y['related'] = Y['related'].map(lambda x: 1 if x == 2 else x)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(t).lower().strip() for t in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipe = Pipeline([('cvet', CountVectorizer(tokenizer=tokenize)),
                    ('tfdif', TfidfTransformer()),
                    ('rfc', RandomForestClassifier())
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipe.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('cvet', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred_train = pipe.predict(X_train['message'])
y_pred_test = pipe.predict(X_test['message'])

In [8]:
print(classification_report(y_test.values, y_pred_test, target_names = Y.columns.values))
print('\n',classification_report(y_train.values, y_pred_train, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.90      0.87      5006
               request       0.84      0.37      0.52      1158
                 offer       0.00      0.00      0.00        28
           aid_related       0.77      0.43      0.56      2718
          medical_help       0.36      0.01      0.02       511
      medical_products       0.58      0.02      0.04       340
     search_and_rescue       0.80      0.02      0.04       187
              security       0.00      0.00      0.00       110
              military       0.60      0.01      0.02       237
                 water       0.80      0.09      0.16       440
                  food       0.86      0.30      0.44       711
               shelter       0.89      0.11      0.20       586
              clothing       0.29      0.02      0.04        99
                 money       0.75      0.02      0.04       152
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {'rfc__max_depth':[5, None],
              'rfc__n_estimators':[15, 50],
              'rfc__min_samples_split':[5, 10],
              'rfc__min_samples_leaf':[1, 5]
             }

cv = GridSearchCV(pipe, param_grid=parameters, scoring='f1_micro', verbose=1, n_jobs=1)

In [11]:
cv.fit(X_train['message'], y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 21.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvet', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'rfc__max_depth': [5, None], 'rfc__n_estimators': [15, 50], 'rfc__min_samples_split': [5, 10], 'rfc__min_samples_leaf': [1, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
y_pred_train_cv = cv.predict(X_train['message'])
y_pred_test_cv = cv.predict(X_test['message'])

In [13]:
print(classification_report(y_test.values, y_pred_test_cv, target_names = Y.columns.values))
print('\n',classification_report(y_train.values, y_pred_train_cv, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.96      0.89      5006
               request       0.88      0.41      0.56      1158
                 offer       0.00      0.00      0.00        28
           aid_related       0.81      0.51      0.62      2718
          medical_help       0.67      0.01      0.02       511
      medical_products       0.50      0.01      0.02       340
     search_and_rescue       1.00      0.01      0.01       187
              security       0.00      0.00      0.00       110
              military       1.00      0.00      0.01       237
                 water       0.93      0.12      0.21       440
                  food       0.89      0.26      0.40       711
               shelter       0.97      0.05      0.09       586
              clothing       0.67      0.02      0.04        99
                 money       0.67      0.01      0.03       152
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
cv.best_params_

{'rfc__max_depth': None,
 'rfc__min_samples_leaf': 1,
 'rfc__min_samples_split': 5,
 'rfc__n_estimators': 50}

In [27]:
pipe2 = Pipeline([('hvet', HashingVectorizer(tokenizer=tokenize)),
                  ('tfidf', TfidfTransformer()),
                  ('rfc1', RandomForestClassifier())
                 ])
pipe2.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('hvet', HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
         decode_error='strict', dtype=<class 'numpy.float64'>,
         encoding='utf-8', input='content', lowercase=True,
         n_features=1048576, ngram_range=(1, 1), non_negative=False,
         norm='l2', pr...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [28]:
y_pred_train2 = pipe2.predict(X_train['message'])
y_pred_test2 = pipe2.predict(X_test['message'])

In [30]:
print(classification_report(y_test.values, y_pred_test2, target_names = Y.columns.values))
print('\n',classification_report(y_train.values,  y_pred_train2, target_names = Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.92      0.87      5006
               request       0.85      0.31      0.45      1158
                 offer       0.00      0.00      0.00        28
           aid_related       0.75      0.40      0.52      2718
          medical_help       0.55      0.01      0.02       511
      medical_products       0.80      0.02      0.05       340
     search_and_rescue       0.50      0.01      0.01       187
              security       0.50      0.01      0.02       110
              military       0.50      0.00      0.01       237
                 water       0.83      0.07      0.12       440
                  food       0.85      0.21      0.33       711
               shelter       0.82      0.04      0.07       586
              clothing       0.50      0.04      0.07        99
                 money       0.40      0.01      0.03       152
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [31]:
model_rfc = pickle.dumps('rfc')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.